In [ ]:
import pyodbc
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Connect to the database
conn_string = "Driver={ODBC Driver 18 for SQL Server};Server=tcp:finance-datacenter.database.windows.net,1433;Database=high-frequency-data;Uid=CloudSA133cd0dc;Pwd=cloud.admin2023;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"

cnx = pyodbc.connect(conn_string)
cursor = cnx.cursor()

In [ ]:
cursor.execute(
    "SELECT DISTINCT(date_time) FROM [dbo].[ive_tickbidask] WHERE date_time >= '2022-01-01 00:00:00.000';"
)
dates = cursor.fetchall()

dates = [date[0] for date in dates]
dates = sorted(list(set([date.date() for date in dates])))

In [ ]:
volatilities = {}
for date in dates:
    cursor.execute(
        "SELECT price FROM [dbo].[ive_tickbidask] WHERE date_time >= '"
        + str(date)
        + " 00:00:00.000' AND date_time <= '"
        + str(date)
        + " 23:59:59.999';"
    )
    data = np.array([x[0] for x in cursor.fetchall()])
    returns = np.diff(data) / data[:-1]
    volatilities[date] = np.sqrt(np.sum(returns**2) / len(returns)) * 100

In [ ]:
cursor.close()
cnx.close()

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(volatilities.keys(), volatilities.values())
plt.title("Daily Volatility of IVE")
plt.xlabel("Date")
plt.ylabel("Volatility")
plt.show()